In [11]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# kala77 andmebaasi tegemine

Skripti töö tulemusena tekitatakse uus andmebaasifail **kala77**.
Baas sisaldab kõiki transaktsioone, mis pole kaetud olemasolevate mustritega.
Kaetud on **ainult mustribaasis olevad** verbid.

Sisendiks on:

- verbi mustrite andmebaas: verb_patterns_new.db
- verbi transaktsioonide andmebaas: v33_transactions.db

## andmebaas kala77

### Tabelid

### TABEL1 transaction_head

| väli          | tüüp | kirjeldus                                                                         | näide      | märkus          |
| ------------- | ---- | --------------------------------------------------------------------------------- | ---------- | --------------- |
| id            | int  | rea <br/>unikaalne ID                                                             | _56_       |                 |
| sentence_id   | int  | lause id andmebaasis                                                              |            |                 |
| loc           | int  | verbi asukoht lauses                                                              |            |                 |
| verb          | text | verbi lemma                                                                       | _olema_    |                 |
| verb_compound | text | verbi afiksaaladverbid                                                            | alla,peale | eraldajaks koma |
| form          | text | verb sellises vormis, nagu see lauses esines                                      | _oli_      |                 |
| deprel        | text | verbi deprel                                                                      |            |                 |
| feats         | text | verbi morf kategooriad alfabeetilises järjekorras                                 | aux,ps3    |                 |
| phrase        | text | puhastatud fraas (ainult need alluvad, mis on transactions tabelisse salvestatud) |            |                 |

### TABEL2 transaction

| väli       | tüüp | kirjeldus                                                            | näide  | märkus |
| ---------- | ---- | -------------------------------------------------------------------- | ------ | ------ |
| id         | int  | rea <br/>unikaalne ID                                                | _56_   |        |
| head_id    | int  | rea transaction_head.id                                              |        |        |
| loc        | int  | sõna asukoht lauses                                                  |        |        |
| loc_rel    | int  | sõna asukoht verbi suhtes                                            |        |        |
| deprel     | text | sõna deprel                                                          |        |        |
| form       | text | sõna vorm                                                            |        |        |
| lemma      | text | sõna lemma                                                           |        |        |
| pos        | text | sõna sõnaliik                                                        |        |        |
| feats      | text | sõna morf kategooriad alfabeetilises järjekorras                     | add,sg |        |
| parent_loc | int  | vanema tipu loc, juhul kui tegemist on <code>obl</obl> alluvaga case | 2      |        |

### TABEL3 verbs_table

| väli          | tüüp | kirjeldus                               | näide    | märkus          |
| ------------- | ---- | --------------------------------------- | -------- | --------------- |
| verb_id       | int  | verbi unikaalne id                      |          |                 |
| verb          | text | verbi lemma                             | _aasima_ |                 |
| verb_compound | text | verbi afiksaaladverbid                  |          | eraldajaks koma |
| pat_ids       | text | mustrite id-d verb_patterns andmebaasis |          | eraldajaks koma |

### TABEL4 verb_transactions

| väli    | tüüp | kirjeldus         | näide | märkus |
| ------- | ---- | ----------------- | ----- | ------ |
| verb_id | int  | verbi id          |       |        |
| head_id | int  | transaktsiooni id |       |        |


In [12]:
import os
from pathlib import Path
from kala77_helpers import *
import sqlite3
from sqlalchemy import create_engine, text


ROOT = str(Path(os.getcwd()).parent.parent)

PATH_TRANSACTIONS_DB = (
    ROOT
    + "/verb_transactions/v33/v33_koondkorpus_sentences_verb_pattern_obl_20241002-130310.db"
)
PATH_PATTERNS_DB = ROOT + "/verb_patterns/vp_data2.db"
KALA77_DB = os.getcwd() + "/kala77.db"

In [13]:
# tekitab andmebaasifaili, kui seda veel ei olnud

con = sqlite3.connect(KALA77_DB)
con.close()


DATABASE_PATH = f"sqlite:///{KALA77_DB}"
print(DATABASE_PATH)

engine = create_engine(DATABASE_PATH)
reset_tables(engine)


# kasutame sqlalchemyt, et oleks lihtsam tabeleid luua
conn = engine.connect()

# liidame teised andmebaasid
conn.execute(text(f"ATTACH DATABASE '{PATH_PATTERNS_DB}' AS db_pat"))
conn.execute(text(f"ATTACH DATABASE '{PATH_TRANSACTIONS_DB}' AS db_tr"))

sqlite:////Users/rabauti/repos/tu/estnltk/syntax_experiments__verb_templates/verb_patterns/kala77/kala77.db


In [14]:
%%time
# täidame verbs tabeli
fill_table_verbs(conn=conn)

CPU times: user 1.65 ms, sys: 854 μs, total: 2.51 ms
Wall time: 2.82 ms


In [15]:
%%time
verbs = conn.execute(select(verbs_table.c.verb_id, verbs_table.c.pat_ids, verbs_table.c.verb, verbs_table.c.verb_compound)).mappings().all()
print(f"andmebaasi lisati {len(verbs)} verbi")

andmebaasi lisati 1271 verbi
CPU times: user 2.01 ms, sys: 605 μs, total: 2.61 ms
Wall time: 2.21 ms


In [16]:
%%time
# täidame verb_transactions tabeli
from tqdm import tqdm
for v in tqdm(verbs):
    fill_table_verb_transactions(conn=conn, verb_id=v['verb_id'], pat_ids=v['pat_ids'].split(','))
conn.execute(text("SELECT COUNT(verb_id) FROM verb_transactions")).scalar()  

100%|██████████| 10/10 [00:00<00:00, 22.18it/s]

CPU times: user 67.2 ms, sys: 55.9 ms, total: 123 ms
Wall time: 454 ms


54686

In [17]:
%%time 
# täidame transaction_head tabeli, võtab ca paarkümmend minutit aega
fill_table_transaction_head(conn=conn)


CPU times: user 146 ms, sys: 239 ms, total: 384 ms
Wall time: 2.67 s


In [18]:
%%time 
# täidame transaction_head tabeli, võtab ca paarkümmend minutit aega
fill_table_transaction(conn=conn)

CPU times: user 5.63 s, sys: 1.19 s, total: 6.82 s
Wall time: 7.97 s


In [19]:
%%time
# kontrollime 10 juhusliku verbi pealt, et numbrid jooksevad kokku
import random
random_i = random.sample(range(0, len(verbs)-1), 10)
# check verbs stat
# get counts of random transactions to check, that numbers align together
for v in [verbs[i] for i in random_i]:
    show_verb_trans_stat(conn=conn, verb=v)
    

{'verb_id': 715, 'pat_ids': '1318', 'verb': 'orjama', 'verb_compound': ''}
db_pat total: 106
db_pat unmatched: 64
db_pat matched: 42
db_tr all: 106
kala77 all: 0
 
{'verb_id': 880, 'pat_ids': '1427', 'verb': 'saama', 'verb_compound': 'pihta'}
db_pat total: 1939
db_pat unmatched: 1108
db_pat matched: 831
db_tr all: 1939
kala77 all: 0
 
{'verb_id': 461, 'pat_ids': '1747', 'verb': 'kukkuma', 'verb_compound': 'sisse'}
db_pat total: 280
db_pat unmatched: 260
db_pat matched: 20
db_tr all: 280
kala77 all: 0
 
{'verb_id': 1059, 'pat_ids': '470', 'verb': 'tulema', 'verb_compound': 'järgi'}
db_pat total: 87
db_pat unmatched: 78
db_pat matched: 9
db_tr all: 87
kala77 all: 0
 
{'verb_id': 1257, 'pat_ids': '2441', 'verb': 'ühenduma', 'verb_compound': ''}
db_pat total: 103
db_pat unmatched: 66
db_pat matched: 37
db_tr all: 103
kala77 all: 0
 
{'verb_id': 888, 'pat_ids': '1634', 'verb': 'sallima', 'verb_compound': ''}
db_pat total: 2820
db_pat unmatched: 871
db_pat matched: 1949
db_tr all: 2820
kala7

KeyboardInterrupt: 

In [10]:
conn.close()